In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_clean=application_df.drop(columns="NAME")
application_clean2=application_clean.drop(columns="EIN")
application_clean3=application_clean.drop(columns="AFFILIATION")
application_df_clean = pd.DataFrame(application_clean3)
application_df_clean.head()

,EIN,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df_clean.nunique()

EIN                       34299
APPLICATION_TYPE             17
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df_clean.value_counts("APPLICATION_TYPE")

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_count[application_count <1000].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df_clean.value_counts("CLASSIFICATION")
classification_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df_clean.value_counts("CLASSIFICATION")
classification_counts_greater1 = classification_counts[classification_counts > 1]
classification_counts_greater1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [8]:

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#classifications_to_replace = ["C1000","C2000","C1200","C1235","C1237","C7210","C1720","C4100","C2400","C1600","C1257","C2710","C1260","C0","C1267","C1246","C1256","C3200","C1234","C4500","C4120","C5200","C3700","C6100","C4200","C1900","C2600","C1728","C1236","C1245","C1248","C1283","C1370","C1570","C1580","C1732","C2570","C1820","C2150","C2170","C2190","C2380","C2500","C2561","C8210"]
classifications_to_replace = classification_count[classification_count <50].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#dummy_columns = ["APPLICATION_TYPE","CLASSIFICATION","AFFILIATION","USE_CASE","ORGANIZATION","STATUS","INCOME_AMT","SPECIAL_CONSIDERATIONS","IS_SUCCESSFUL"]
application_df_dummies = pd.get_dummies(application_df_clean)
application_df_dummies.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10531628,1,108590,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,10547893,1,5000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,10553066,1,6692,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,10556103,1,142590,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies["IS_SUCCESSFUL"]
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization Attempt #1

Dropped: Affiliation colum
Increasing or decreasing the number of values for each bin. Changed the threshold for application types to 1000, and classification column to 50

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=70, activation='relu', input_dim=input_features))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn1.summary()

# Compile the model
nn1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn1.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Optimization Attempt #2

Add more neurons to the hidden layer.
and added more hidden layers.

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=40, activation='relu'))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

# Compile the model
nn2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn2.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3680      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 20)                820       
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 7761 (30.32 KB)
Trainable params: 7761 (30.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 4s 2ms/step - loss: 0.6253 - accuracy: 0.6426
Epoch 2/100
804/804 [==============================

Optimization Attempt #3

Use different activation functions for the hidden layers, and added the number of epochs to the training regimen.

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation='tanh', input_dim=input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=40, activation='tanh'))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=20, activation='tanh'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

# Compile the model
nn3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn3.fit(X_train_scaled,y_train,epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3680      
                                                                 
 dense_8 (Dense)             (None, 40)                3240      
                                                                 
 dense_9 (Dense)             (None, 20)                820       
                                                                 
 dense_10 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7761 (30.32 KB)
Trainable params: 7761 (30.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6220 - accuracy: 0.6447
Epoch 2/200
804/804 [==============================

I tried to figure out the accuracy of the "Is_Successful" variable
The features for this model started as the variables in all columns other than 'Is_Succesful'. I also cleaned  non-benefitial columns ('EIN' and 'Name') to not be included as features. Also, as part of my first optimization attempt I removed 'Affiliation' from features.
I removed acoulpe features that i thought they were non-beneficial.

 I started with 70 neurons in the first hidden layer as there were 69 columns. My original model leveraged ReLu (Rectified Linear Unit) as the activation function for the first two layers as it is the most commonly used activation function used in nueral networks. Then for the output layer I chose to use the Sigmoid function as this is also good for classifications.

no i wasnt able to achieve the optimial 85% accuracy. the highest i got was 66%. for each optimization i tried to tweak the columns, the layers, and neurons. but my change didnmt do much on the accuracy tests. i got 60-66% on the three tries.
